In [1]:
import pandas as pd
import osmnx as ox

CIDADE = "Recife, Pernambuco, Brasil"
ARQUIVO = "bairros_vizinhos.csv"
GEO_PACKAGE = "grafo_recife.graphml"
SAIDA = "bairros_vizinhos_tratados.csv"

mapeamento_pesos = {
    "motorway": 0.8,
    "motorway_link": 0.8,
    "trunk": 1.0,
    "trunk_link": 1.0,
    "primary": 1.2,
    "primary_link": 1.2,
    "secondary": 1.5,
    "secondary_link": 1.5,
    "tertiary": 1.8,
    "tertiary_link": 1.8,
    "residential": 2.0,
    "service": 2.5,
    "living_street": 2.5,
    "pedestrian": 3.0,
    "track": 3.0,
}

tipo_normalizado = {
"motorway":	"Via expressa / Rodovia",
"motorway_link": "Alça de acesso à via expressa",
"trunk": "Via arterial / Rodovia urbana",
"trunk_link": "Acesso à via arterial",
"primary":	"Avenida principal",
"primary_link":	"Alça de acesso à avenida principal",
"secondary": "Avenida secundária	",
"secondary_link": "Ramal de ligação secundária",
"tertiary": "Rua de ligação / Coletora local",
"tertiary_link": "Trecho curto entre ruas terciárias",
"residential": "Rua local / de bairro",
"service": "Rua de serviço / interna",
"living_street": "Rua compartilhada / de pedestres",
"pedestrian": "Calçadão / rua exclusiva de pedestres",
"track": "Estrada de terra / rural",
}

def normalizar_tipo(tipo):
    if isinstance(tipo, list):
        prioridade = ["motorway", "trunk", "primary", "secondary", "tertiary", "residential", "service"]
        for p in prioridade:
            if p in tipo:
                return p
        return tipo[0]
    if "_link" in tipo:
        return tipo.replace("_link", "")
    return tipo




In [2]:
# === Lê os pares de bairros ===
df = pd.read_csv(ARQUIVO, sep=",")
df.head()


,name,neighbor
0,Aflitos,Rosarinho
1,Aflitos,Encruzilhada
2,Aflitos,Espinheiro
3,Aflitos,Graças
4,Jaqueira,Tamarineira


In [3]:
print("Carregando mapa da cidade a partir do OSM...")
G = ox.graph_from_place(CIDADE, network_type="drive")


Carregando mapa da cidade a partir do OSM...


In [ ]:
resultados = []
pares_processados = set()
usados = set()

total_size = len(df)
print(f"Total de pares a processar: {total_size}")

for index, row in df.iterrows():
    bairro_a = row["name"]
    bairro_b = row["neighbor"]
    print(f"Processando par {index + 1} de {total_size}: {bairro_a} ↔ {bairro_b}")
    
    par_ordenado = tuple(sorted([bairro_a, bairro_b]))
    if par_ordenado in pares_processados:
        continue
    pares_processados.add(par_ordenado)

    try:
        orig_node = ox.distance.nearest_nodes(G, *ox.geocode(f"{bairro_a}, {CIDADE}")[::-1])
        dest_node = ox.distance.nearest_nodes(G, *ox.geocode(f"{bairro_b}, {CIDADE}")[::-1])

        route = ox.routing.shortest_path(G, orig_node, dest_node, weight="length")

        for nodo_origem, nodo_destino in zip(route[:-1], route[1:]):
            atributos_aresta = G.edges[nodo_origem, nodo_destino, 0]
            nome = atributos_aresta.get("name")
            tipo = atributos_aresta.get("highway")
            osmid = atributos_aresta.get("osmid")

            # Normaliza caso sejam listas
            if isinstance(nome, list):
                nome = nome[0]
            if isinstance(tipo, list):
                tipo = tipo[0]
            if isinstance(osmid, list):
                osmid = osmid[0]

            # Pega o primeiro nome ainda não usado
            if nome and nome not in usados:
                nome_selecionado = nome
                tipo_selecionado = tipo
                id_rua = osmid
                usados.add(nome)
                break

        if nome_selecionado is None:
            nome_selecionado = "N/A"
            tipo_selecionado = "N/A"
            id_rua = "N/A"

        resultados.append({
            "Bairro": bairro_a,
            "Vizinho": bairro_b,
            "Logradouro": nome_selecionado,
            "Tipo": tipo_selecionado,
            "Peso": mapeamento_pesos.get(normalizar_tipo(tipo_selecionado), 2.0),
            "Tipo Normalizado": tipo_normalizado.get(normalizar_tipo(tipo_selecionado), "Desconhecido"),
            "Id Rua": f"https://openstreetmap.org/way/{id_rua}"
        })

    except Exception as e:
        print(f"Erro ao processar {bairro_a}-{bairro_b}: {e}")
        resultados.append({
            "Bairro": bairro_a,
            "Vizinho": bairro_b,
            "Logradouro": "N/A",
            "Tipo": "N/A",
            "Peso": "N/A",
            "Tipo Normalizado": "N/A",
            "Id Rua": "N/A"
        })


Total de pares a processar: 494
Processando par 1 de 494: Aflitos ↔ Rosarinho
dict_keys(['osmid', 'highway', 'lanes', 'maxspeed', 'name', 'oneway', 'reversed', 'length'])
Processando par 2 de 494: Aflitos ↔ Encruzilhada
dict_keys(['osmid', 'highway', 'lanes', 'maxspeed', 'name', 'oneway', 'reversed', 'length'])
dict_keys(['osmid', 'highway', 'lanes', 'name', 'oneway', 'reversed', 'length', 'geometry'])
Processando par 3 de 494: Aflitos ↔ Espinheiro
dict_keys(['osmid', 'highway', 'lanes', 'maxspeed', 'name', 'oneway', 'reversed', 'length'])
dict_keys(['osmid', 'highway', 'lanes', 'name', 'oneway', 'reversed', 'length', 'geometry'])
dict_keys(['osmid', 'highway', 'name', 'oneway', 'reversed', 'length'])
Processando par 4 de 494: Aflitos ↔ Graças
dict_keys(['osmid', 'highway', 'lanes', 'maxspeed', 'name', 'oneway', 'reversed', 'length'])
dict_keys(['osmid', 'highway', 'lanes', 'name', 'oneway', 'reversed', 'length', 'geometry'])
Processando par 5 de 494: Jaqueira ↔ Tamarineira
dict_keys([

In [ ]:
df_result = pd.DataFrame(resultados)
df_result.to_csv(SAIDA, index=False)
print(f"Resultado salvo em {SAIDA}")
df_result.head()


✅ Resultado salvo em bairros_vizinhos_tratados.csv


,Bairro,Vizinho,Logradouro,Tipo,Peso,Tipo Normalizado,Id Rua
0,Aflitos,Rosarinho,Rua Manuel de Carvalho,tertiary,1.8,Rua de ligação / Coletora local,https://openstreetmap.org/way/134174443
1,Aflitos,Encruzilhada,Rua Teles Júnior,tertiary,1.8,Rua de ligação / Coletora local,https://openstreetmap.org/way/134167460
2,Aflitos,Espinheiro,Rua Gomes de Matos Júnior,residential,2.0,Rua local / de bairro,https://openstreetmap.org/way/134167725
3,Aflitos,Graças,Rua Samuel Campelo,tertiary,1.8,Rua de ligação / Coletora local,https://openstreetmap.org/way/966042500
4,Jaqueira,Tamarineira,Rua Neto de Mendonça,tertiary,1.8,Rua de ligação / Coletora local,https://openstreetmap.org/way/965984036


In [6]:
bairros_finais = set(df_result['Bairro'])

bairros_originals = set(df['name'])

diferenca = bairros_originals - bairros_finais
print(len(diferenca))
for bairro in diferenca:
    print("Bairro não encontrado na primeira coluna:", bairro)
    
bairros_finais = set(df_result['Vizinho'])

bairros_originals = set(df['neighbor'])

diferenca = bairros_originals - bairros_finais
print(len(diferenca))
for bairro in diferenca:
    print("Bairro não encontrado na segunda coluna:", bairro)


9
Bairro não encontrado na primeira coluna: Poço da Panela
Bairro não encontrado na primeira coluna: Soledade
Bairro não encontrado na primeira coluna: Parnamirim
Bairro não encontrado na primeira coluna: Zumbi
Bairro não encontrado na primeira coluna: Ipsep
Bairro não encontrado na primeira coluna: Oitenta
Bairro não encontrado na primeira coluna: Mangabeira
Bairro não encontrado na primeira coluna: Santo Antônio
Bairro não encontrado na primeira coluna: Água Fria
16
Bairro não encontrado na segunda coluna: Hipódromo
Bairro não encontrado na segunda coluna: Beberibe
Bairro não encontrado na segunda coluna: Coqueiral
Bairro não encontrado na segunda coluna: Bairro do Recife
Bairro não encontrado na segunda coluna: Cajueiro
Bairro não encontrado na segunda coluna: Joana Bezerra
Bairro não encontrado na segunda coluna: Caxangá
Bairro não encontrado na segunda coluna: Aflitos
Bairro não encontrado na segunda coluna: Alto José do Pinho
Bairro não encontrado na segunda coluna: Boa Viagem
Ba